# Исследование надежности заемщиков

## Изучение общей информации о данных

**Путь к файлу с данными: `/datasets/data.csv`**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца, используя комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
data['children'].unique() #смотрим, какие есть значения в колонке 'children' 

array([1, 0, 3, 2, 4, 5])

In [23]:
#агрегация 'children' и 'debt'
children_debt_grouped = data.groupby('children', as_index=False).agg({'debt': ['count', 'sum']})
#расчет доли должников среди всех людей с детьми или без детей
children_debt_grouped['ratio_children_percent'] = children_debt_grouped['debt']['sum'] / children_debt_grouped['debt']['count']
#приведение к визуально понятному отражению доли должников
children_debt_grouped['ratio_children_percent'] = children_debt_grouped['ratio_children_percent'].multiply(100).round(2)
#сортировка от меньшего к большему
children_debt_grouped.sort_values('ratio_children_percent', ascending = True, inplace =True)
display(children_debt_grouped)

children   debt       ratio_children_percent
            count   sum                       
5        5      9     0                   0.00
0        0  14107  1063                   7.54
3        3    330    27                   8.18
1        1   4809   444                   9.23
2        2   2052   194                   9.45
4        4     41     4                   9.76

**Вывод:** 

По значениям в колонке ratio_children_percent можно увидеть, что процент должников без детей незначительно меньше, чем процент должников с детьми, однако среди людей с разным количеством детей нет большой разницы в платежеспособности. Самый низкий процент должников наблюдается у людей с тремя детьми, но такой результат мог появиться из-за нерепрезантивной выборки (всего 330 человек по сравнению с 4809 людей с одним ребенком). 

Итог: не наблюдается явной зависимости между количеством детей и возвратом кредита в срок, но вероятность возвращения кредита в срок немного выше у бездетных заемщиков.

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [24]:
data['family_status'].unique() #смотрим, какие есть значения в колонке 'family_status_id' 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [25]:
family_status_debt_grouped = data.groupby('family_status', as_index = False).agg({'debt': ['count', 'sum']})
family_status_debt_grouped['ratio_family_status_percent'] = family_status_debt_grouped['debt']['sum'] / family_status_debt_grouped['debt']['count']
family_status_debt_grouped['ratio_family_status_percent'] = family_status_debt_grouped['ratio_family_status_percent'].multiply(100).round(2)
family_status_debt_grouped.sort_values('ratio_family_status_percent', ascending = True, inplace = True)
display(family_status_debt_grouped)

family_status   debt      ratio_family_status_percent
                          count  sum                            
2         вдовец / вдова    951   63                        6.62
1              в разводе   1189   84                        7.06
4        женат / замужем  12266  927                        7.56
3       гражданский брак   4146  385                        9.29
0  Не женат / не замужем   2796  273                        9.76

**Вывод:** 

По значениям в колонке ratio_family_status_percent можно увидеть, что процент должников среди тех, кто не в браке или состоят в неофициальных отношениях выше, чем среди женатых/замужних заемщиков и заемщиков в разводе. Самый низкий процент должников наблюдается среди вдовцов/вдов, но это результат опять же может быть вызван нерепрезентативностью выборки (всего 951 человек). 

Итог: наблюдается зависимость между семейным положением и возвратом кредита в срок.

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [26]:
#Обратимся к колонке 'total_income_category', где ранее были выделены категории по уровню дохода

total_income_debt_grouped = data.groupby('total_income_category', as_index = False).agg({'debt': ['count', 'sum']})
total_income_debt_grouped['ratio_total_income_percent'] = total_income_debt_grouped['debt']['sum'] / total_income_debt_grouped['debt']['count']
total_income_debt_grouped['ratio_total_income_percent'] = total_income_debt_grouped['ratio_total_income_percent'].multiply(100).round(2)
total_income_debt_grouped.sort_values('ratio_total_income_percent', ascending = True, inplace = True)
display(total_income_debt_grouped)

total_income_category   debt       ratio_total_income_percent
                         count   sum                           
3                     D    349    21                       6.02
1                     B   5014   354                       7.06
0                     A     25     2                       8.00
2                     C  15938  1353                       8.49
4                     E     22     2                       9.09

In [27]:
#Напомним значения групп
print("0–30000 — 'E'\n30001–50000 — 'D'\n50001–200000 — 'C'\n200001–1000000 — 'B'\n1000001 и выше — 'A'.")

0–30000 — 'E'
30001–50000 — 'D'
50001–200000 — 'C'
200001–1000000 — 'B'
1000001 и выше — 'A'.


**Вывод:** 

В выборке представлено всего 22 человека из категории Е и 25 человек категории А, и несмотря на то, что, казалось бы, чем выше доход, тем меньше вероятность невозврата кредита в срок, категория А не попала на последнее место, следовательно, на основе данной выборки нельяз сделать вывод о наличии зависимости. Обратимся к категориям, которые содержат больше данных. Мы можем увидеть, что среди заемщиков категории С процент должников выше, чем среди заемщиков категории В, однако категория D демонстрирует наименьший процент должников, хотя это категория людей с не самым высоким достатком. 

Итог: явной зависимости между уровнем дохода и возвратом кредита в срок в данной выборке не наблюдается, но среди наиболее полно представленных категорий В и С, в категории В процент должников ниже, следовательно, у людей с более высоким достатков больше вероятность возврата кредита в срок, чем с более низким.

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [28]:
#Обратимся к колонке 'purpose_category', где ранее были выделены категории по целям взятия кредита

purpose_debt_grouped = data.groupby('purpose_category', as_index = False).agg({'debt': ['count', 'sum']})
purpose_debt_grouped['ratio_purpose_percent'] = purpose_debt_grouped['debt']['sum'] / purpose_debt_grouped['debt']['count']
purpose_debt_grouped['ratio_purpose_percent'] = purpose_debt_grouped['ratio_purpose_percent'].multiply(100).round(2)
purpose_debt_grouped.sort_values('ratio_purpose_percent', ascending = True, inplace = True)
display(purpose_debt_grouped)

purpose_category   debt      ratio_purpose_percent
                             count  sum                      
1  операции с недвижимостью  10754  780                  7.25
3        проведение свадьбы   2324  183                  7.87
2     получение образования   3989  369                  9.25
0    операции с автомобилем   4281  400                  9.34

**Вывод:** 

По значениям в колонке ratio_purpose_percent можно увидеть, что при займе для операций с автомобилем и получения образования наблюдается наибольший процент должников. Меньше должников наблюдается среди людей, целью кредита которых была операция с недвижимостью и проведение свадьбы.

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* Значения пропущены в колонках days_employed и total_income и их количество совпадает, следовательно, можно предположить, что заемщики у которых нет трудового стажа также и не получают доход, поэтому данные пропущены.

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* В выборке присутствуют выдающиеся значения, поэтому лучше использовать медиану, которая более устойчива к «выбросам», а также можно использовать моду.

## Общий вывод.

При анализе и агрегации данных были сделаны следующие выводы:

1. Не наблюдается явной зависимости между количеством детей и возвратом кредита в срок, но вероятность возвращения кредита в срок немного выше у бездетных заемщиков;
2. Процент должников среди тех, кто не в браке или состоят в неофициальных отношениях выше, чем среди женатых/замужних заемщиков и заемщиков в разводе. Самый низкий процент должников наблюдается среди вдовцов/вдов;
3. Явной зависимости между уровнем дохода и возвратом кредита в срок в данной выборке не наблюдается, однако у людей с более высоким достатков немного больше вероятность возврата кредита в срок, чем с более низким;
4. При займе для операций с автомобилем и получения образования наблюдается наибольший процент должников. Меньше должников наблюдается среди людей, целью кредита которых была операция с недвижимостью и проведение свадьбы.